In [37]:
from snownlp import SnowNLP as sn
import pandas as pd
import jieba
import sys
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer

jieba.set_dictionary('dict.txt.big.txt')#繁體詞庫
#http://bruce3557.logdown.com/posts/291096/jieba-sklearn-calculation-of-chinese-tfidf

- 標點刪除函式


In [38]:
text = "*/@》--【】--12()测试*()"

'''去除字符串中所有的字符，可增加自定义字符'''
def strclear(text,newsign=''):
    import string # 引入string模块
    signtext = string.punctuation + newsign # 引入英文符号常量，可附加自定义字符，默认为空
    signrepl = '@'*len(signtext) # 引入符号列表长度的替换字符
    signtable = str.maketrans(signtext,signrepl) # 生成替换字符表
    return text.translate(signtable).replace('@','') # 最后将替换字符替换为空即可

strclear(text,'》【】')

'12测试'

# tfidf

- 讀檔&需要刪除的字、標點符號


In [152]:
f = open('4.txt', 'r') 
ddd = f.read()
d1 = ''.join(f.read().split()) #刪除所有標點、空白、換行...
stopwords = ['\n'] #不想要的字詞


UnicodeDecodeError: 'cp950' codec can't decode byte 0x84 in position 69207: illegal multibyte sequence

In [153]:
ddd = f.read()
ddd

''

- 斷詞

In [41]:
d1 = jieba.cut(d1, cut_all=False) #generator

- 擷取需要的詞並用空白座分隔不同文章

In [43]:
d1 = list(d1)
corpus = []
for w in d1:
    if  w not in stopwords:        
        corpus.append(w)
        
corpus.append(" ")


- 讀取每一份(寫成函式形式)


In [157]:
def read_append(n,corpus):     #n:第幾份；corpus:目前資料
    f = open(str(n)+'.txt', 'r') 
    try:
        d1 = strclear(''.join(f.read().split())) #存取並刪除所有標點、空白、換行...        
        f.close()
        d1 = ' '.join(jieba.cut(d1, cut_all=False)) #generator
        corpus.append(d1)
    except:
     #   d1 = strclear(''.join(f.read().split())) #存取並刪除所有標點、空白、換行... 
     #   f.close()
        print('error at '+str(n)+'.txt') 
    return corpus

- 總共1-6份

In [158]:
corpus = []
for n in range(1,7):    
    corpus= read_append(n,corpus)

error at 3.txt
error at 4.txt


- 建立vectorizer (共4份文件, 7408個詞,2份錯誤)

In [159]:
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(corpus)
print (tfidf.shape)

(4, 7408)


- 將tfidf >1e-2 的 詞作排序

In [168]:
words_=[]
values_=[]

In [169]:
dict_tfidf = {}
words = vectorizer.get_feature_names()
for i in range(len(corpus)):
   # print ('----Document '+ str(i) + '----' )
    for j in range(len(words)):
        if tfidf[i,j] > 1e-5:
            words_.append(words[j])
            values_.append(tfidf[i,j])

In [170]:
frame_word = pd.Series(words_)
frame_values = pd.Series(values_)
data = {'words':frame_word,'value':frame_values}
frame = pd.DataFrame(data)
important_word = frame.sort_values(by=['value'], ascending=False)
important_word

,value,words
7103,0.557219,公司
3166,0.534255,公司
12625,0.521829,公司
881,0.385819,公司
1026,0.292447,合併
13513,0.280187,年度
7973,0.246514,年度
3711,0.240454,年度
855,0.193790,兆利
1412,0.188505,有限公司
